In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stanford-plato-corpus/1-make-data-csv.ipynb
/kaggle/input/stanford-plato-corpus/make-final-plato-corpus.ipynb
/kaggle/input/stanford-plato-corpus/get-data-for-title_copy.ipynb
/kaggle/input/stanford-plato-corpus/data_per_paragraph.csv
/kaggle/input/stanford-plato-corpus/simple_data.csv
/kaggle/input/stanford-plato-corpus/plato_mirror_spr2020.tgz
/kaggle/input/stanford-plato-corpus/get-data-for-title.ipynb
/kaggle/input/stanford-plato-corpus/2-explode-nested-data.ipynb
/kaggle/input/stanford-plato-corpus/sample_summaries_per_document.csv
/kaggle/input/stanford-plato-corpus/corpus_article_text_features.csv
/kaggle/input/stanford-plato-corpus/create-summaries-csv-per-paragraph.ipynb
/kaggle/input/stanford-plato-corpus/data.csv
/kaggle/input/stanford-plato-corpus/corpus_articles.csv


In [2]:
import csv


In [3]:
df = pd.read_csv('/kaggle/input/stanford-plato-corpus/data_per_paragraph.csv')


In [4]:
unique_titles_df = pd.read_csv('/kaggle/input/stanford-plato-corpus/corpus_articles.csv')

sample_10 = unique_titles_df[['filename', 'title']].sample(n=10)

In [5]:
sample_10

,filename,title
190,stanford.library.sydney.edu.au/entries/cancer/...,Cancer
737,stanford.library.sydney.edu.au/entries/infinit...,Infinite Regress Arguments
159,stanford.library.sydney.edu.au/entries/bohr-co...,Bohr’s Correspondence Principle
756,stanford.library.sydney.edu.au/entries/interna...,International Distributive Justice
253,stanford.library.sydney.edu.au/entries/color/i...,Color
93,stanford.library.sydney.edu.au/entries/aristot...,Commentators on Aristotle
1315,stanford.library.sydney.edu.au/entries/qm-retr...,Retrocausality in Quantum Mechanics
892,stanford.library.sydney.edu.au/entries/locke-f...,Locke On Freedom
163,stanford.library.sydney.edu.au/entries/boolalg...,The Mathematics of Boolean Algebra
1281,stanford.library.sydney.edu.au/entries/proof-t...,Proof Theory


In [6]:
x_df = pd.read_csv('/kaggle/input/stanford-plato-corpus/sample_summaries_per_document.csv')

In [7]:
x_df.head()

,Unnamed: 0,title,filename,preamble,data
0,0,Paul of Venice,stanford.library.sydney.edu.au/entries/paul-ve...,Paul of Venice was the most important Italia...,"[{(1.0, 1.0): 'Paul of Venice was born in Udin..."
1,1,"Cusanus, Nicolaus [Nicolas of Cusa]",stanford.library.sydney.edu.au/entries/cusanus...,Arguably the most important German thinker ...,"[{(1.0, 1.0): 'Nicholas of Cusa (Nikolaus Cryf..."


In [8]:
import transformers
from transformers import pipeline
import re
summarizer = pipeline('summarization')


def _with_counter(fun, _N, _i=1):
    """
    print a progress count with each function call
    """
    global COUNTER_I, COUNTER_N
    # initialize for reuse within _fun
    COUNTER_I = _i
    COUNTER_N = _N
    def _fun(*args, **kwargs):
        global COUNTER_I, COUNTER_N
        print(
            f"{COUNTER_I/COUNTER_N}\t == \t{COUNTER_I} / {COUNTER_N}", 
            end="\r"
        )
        print("DONE",f"{COUNTER_I} / {COUNTER_N}","\t"*30) if COUNTER_I >= COUNTER_N else None
        COUNTER_I += 1
        return fun(*args, **kwargs)
    return _fun


def summarize(text, max_length=512, min_length=100):
    key='summary_text'
    summary_list = summarizer(text, max_length=max_length, min_length=min_length)
    summary = summary_list[0][key]
    return summary

# summaries = paragraphs.apply(csummarize)
def simple_preprocess(text):
    text = text.replace('\n', ' ') #Remove line breaks
    matches = re.findall(r'\(([^()]*)\)', text)  #Remove citations
    for i in matches:
        if re.search(r'[0-9][0-9][0-9][0-9]?', i):
            text = text.replace('({})'.format(i), '')
        elif len(i) < 20:
            text = text.replace('({})'.format(i), '')
    return text


#usage get_data(title)
#returns list of dictionaries, preamble
#where each dictionary has only one key and value, key is a tuple like (section_id, paragraph_id) and value is the paragraph_text. 

def get_data(title):
    df['preamble_text'] = df['preamble_text'].fillna(str(''))
    df['section.paragraph.text'] = df['section.paragraph.text'].fillna(str(''))
    if title in list(df['title']):
        req_df = df.loc[df['title'] == title]
        req_df['section.processed_paragraph.text'] = req_df['section.paragraph.text'].apply(simple_preprocess)
        csummarize = _with_counter(summarize, _N=len(req_df))

        req_df['summary'] = req_df['section.processed_paragraph.text'].apply(csummarize)
        data = [{(s,p_id):str(p)} 
                            for s,p_id,p in zip(req_df['section.id'], 
                                                req_df['section.paragraph.id'],
                                                req_df['summary'])]
#         data = ''
        preamble = list(req_df['preamble_text'])[0]
#         print(preamble)
#         if preamble is None:
#             return ''
#         else:
        preamble = simple_preprocess(preamble)
        return data, preamble
    

In [9]:
# x_df = pd.DataFrame(columns=['title', 'filename', 'preamble', 'data'])

In [10]:
x_df.head()

,Unnamed: 0,title,filename,preamble,data
0,0,Paul of Venice,stanford.library.sydney.edu.au/entries/paul-ve...,Paul of Venice was the most important Italia...,"[{(1.0, 1.0): 'Paul of Venice was born in Udin..."
1,1,"Cusanus, Nicolaus [Nicolas of Cusa]",stanford.library.sydney.edu.au/entries/cusanus...,Arguably the most important German thinker ...,"[{(1.0, 1.0): 'Nicholas of Cusa (Nikolaus Cryf..."


In [11]:
data_list = list()
preamble_list = list()
title_list = list()
filename_list = list()
# with open('/sample_summaries_per_document.csv', 'a') as f:
# writer = csv.writer(f)
# writer.writerow(['title', 'filename', 'preamble', 'data'])

for i in range(len(list(sample_10))):
    print('Processing article: {}/9'.format(i))
    data, preamble = get_data(list(sample_10['title'])[i])
    data_list.append(data)
    filename_list.append(list(sample_10['filename'])[i])
    title_list.append(list(sample_10['title'])[i])
    preamble_list.append(preamble)
    x_df = x_df.append({'title':list(sample_10['title'])[i], 'filename':list(sample_10['filename'])[i], 'preamble':preamble, 'data':data}, ignore_index=True)
#         writer = csv.writer(f)
#         writer.writerow([list(sample_10['title'])[i], list(sample_10['filename'])[i], preamble, data])
    

Processing article: 0/9


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


DONE 62 / 62 																														


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Processing article: 1/9
DONE 83 / 83 																														


In [12]:
x_df.to_csv('sample_summaries_per_document.csv')

In [13]:
pd.read_csv('sample_summaries_per_document.csv').head()

,Unnamed: 0,Unnamed: 0.1,title,filename,preamble,data
0,0,0.0,Paul of Venice,stanford.library.sydney.edu.au/entries/paul-ve...,Paul of Venice was the most important Italia...,"[{(1.0, 1.0): 'Paul of Venice was born in Udin..."
1,1,1.0,"Cusanus, Nicolaus [Nicolas of Cusa]",stanford.library.sydney.edu.au/entries/cusanus...,Arguably the most important German thinker ...,"[{(1.0, 1.0): 'Nicholas of Cusa (Nikolaus Cryf..."
2,2,NaN,Cancer,stanford.library.sydney.edu.au/entries/cancer/...,Cancer is a worldwide epidemic. It is the f...,"[{(1.0, 1.0): ' defining and classifying cance..."
3,3,NaN,Infinite Regress Arguments,stanford.library.sydney.edu.au/entries/infinit...,An infinite regress is a series of appropri...,"[{(1.0, 1.0): 'Sometimes it is uncontroversial..."


In [14]:
print(filename_list)
print(title_list)

['stanford.library.sydney.edu.au/entries/cancer/index.html', 'stanford.library.sydney.edu.au/entries/infinite-regress/index.html']
['Cancer', 'Infinite Regress Arguments']


In [15]:
!ls


__notebook__.ipynb  sample_summaries_per_document.csv


In [16]:
# data, preamble = get_data(title)xds


In [17]:
# data

In [18]:

# preamble